<a href="https://colab.research.google.com/github/harrymkwn/InfluenceAnalysis/blob/master/Emointvecmod2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from gensim.models import Word2Vec
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from gensim.models import KeyedVectors
from sklearn.model_selection import train_test_split
from google.colab import  drive
drive.mount('/content/drive')
import nltk
from nltk.corpus import stopwords
nltk.download('all')
device_name = tf.test.gpu_device_name()
print(device_name)
df = pd.read_csv('/content/drive/MyDrive/InfluenceAnalysis/Data/Tweets_clean_data.csv')
df = df.sample(frac=1)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package biocreative_ppi is already up-to-date!
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Package brown_tei is already up-to-date!
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Package cess_cat is already up-to-date!
[nltk_data]    | Downloading pack

In [ ]:
category_dict = {"anger":[1,0,0,0],"sadness":[0,1,0,0],"fear":[0,0,1,0],"joy":[0,0,0,1]}
data_tweet = [x.lower().split() for x in df['Tweet']]
data_cat = np.array([category_dict[x] for x in df['Emotion']])

In [ ]:
print(data_tweet[:5])
print(data_cat[:5])

[['why', 'should', 'i', 'listen', 'to', 'someone', 'with', 'a', 'tie', 'like', 'that'], ['do', 'not', 'let', 'the', 'behavior', 'of', 'others', 'destroy', 'ur', 'inner', 'peace', 'dalai', 'lama', 'healing', 'depression', 'anxiety', 'intuition', 'meditation', 'bok'], ['waltz', 'with', 'bashir', 'was', 'incredible', 'tho', 'i', 'think', 'it', 'is', 'more', 'of', 'an', 'animated', 'film', 'than', 'documentary', 'about', 'lebanon', 'war'], ['michelle', 'who', 'did', 'nothing', 'is', 'hating', 'on', 'nicole', 's', 'game', 'hahaha', 'bitter', 'bb18'], ['just', 'joined', 'pottermore', 'and', 'was', 'sorted', 'into', 'hufflepuff', 'pouting', 'facepouting', 'facepouting', 'face']]
[[0 0 1 0]
 [0 1 0 0]
 [0 0 0 1]
 [1 0 0 0]
 [1 0 0 0]]


In [ ]:
# Parameters
Min_count = 0
Embedding_size = 100
Window_size = 5
Negative_sampling = 00

In [ ]:
w2v_sg = Word2Vec(min_count=Min_count,
                     window=Window_size,
                     size=Embedding_size,
                     negative=Negative_sampling,sg=1)
w2v_sg.build_vocab(data_tweet)
w2v_sg.train(data_tweet, total_examples=w2v_sg.corpus_count, epochs=5)
w2v_cbow = Word2Vec(min_count=Min_count,
                     window=Window_size,
                     size=Embedding_size,
                     negative=Negative_sampling,sg=0)
w2v_cbow.build_vocab(data_tweet)
w2v_cbow.train(data_tweet, total_examples=w2v_cbow.corpus_count, epochs=5)

(475486, 595345)

In [ ]:
vocab_sg = w2v_sg.wv.vocab
vocab_sg = [x for x in vocab_sg]
vocab_cbow = w2v_cbow.wv.vocab
vocab_cbow = [x for x in vocab_cbow] 
vocab_glove = {}
with open("/content/drive/My Drive/InfluenceAnalysis/glove/glove.twitter.27B.100d.txt", 'r', encoding="utf-8") as f:
  for line in f:
      values = line.split()
      word = values[0]
      vector = np.asarray(values[1:], "float32")
      vocab_glove[word] = vector

In [ ]:
batch_size = 100
Max_input_size = max([len(x) for x in data_tweet])
mul_factor = 1.5

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data_tweet, data_cat, test_size=0.30, random_state=42)
X_train = [nltk.pos_tag(x) for x in X_train]
X_test = [nltk.pos_tag(x) for x in X_test]
result_table = [0,0,0,0,0,0,0,0,0]

In [ ]:
def vec_gen(w2v,vocab,data_tweet,data_cat,batch_size,Max_input_size,Embedding_size,mul_factor):
  while True:
    for k in range(int(len(data_tweet)/batch_size)):
      res=[]
      for x in data_tweet[k*(batch_size):(k+1)*(batch_size)]:
        tweet = [(w2v.wv.get_vector(i[0]),i[1]) for i in x if i[0] in vocab]
        l=len(tweet)
        for i in range(l):
          if tweet[i][1][:2]=='JJ':
            for z in range(max(i-2,0),min(i+3,l)):
              if tweet[z][1][:2]=='NN':
                tweet[z]=((tweet[z][0]*tweet[i][0])*mul_factor/np.sqrt(tweet[i][0].dot(tweet[i][0])),tweet[z][1])
          if tweet[i][1][:2]=='RB':
            for z in range(max(i-2,0),min(i+3,l)):
              if tweet[z][1][:2]=='VB':
                tweet[z]=((tweet[z][0]*tweet[i][0])*mul_factor/np.sqrt(tweet[i][0].dot(tweet[i][0])),tweet[z][1])
        res+=[np.array([x[0] for x in tweet])]
      temp = np.array([np.pad(z.flatten(),(0,Max_input_size*Embedding_size-len(z.flatten()))).reshape(Max_input_size,Embedding_size) for z in res])
      tempres = data_cat[k*(batch_size):(k+1)*(batch_size)]
      yield (temp,tempres)          
      
def glove_gen(vocab,data_tweet,data_cat,batch_size,Max_input_size,Embedding_size,mul_factor):
  while True:
    for k in range(int(len(data_tweet)/batch_size)):
      res=[]
      for x in data_tweet[k*(batch_size):(k+1)*(batch_size)]:
        tweet = [(vocab[i[0]],i[1]) for i in x if i[0] in vocab.keys()]
        l=len(tweet)
        for i in range(l):
          if tweet[i][1][:2]=='JJ':
            for z in range(max(i-2,0),min(i+3,l)):
              if tweet[z][1][:2]=='NN':
                tweet[z]=((tweet[z][0]*tweet[i][0])*mul_factor/np.sqrt(tweet[i][0].dot(tweet[i][0])),tweet[z][1])
          if tweet[i][1][:2]=='RB':
            for z in range(max(i-2,0),min(i+3,l)):
              if tweet[z][1][:2]=='VB':
                tweet[z]=((tweet[z][0]*tweet[i][0])*mul_factor/np.sqrt(tweet[i][0].dot(tweet[i][0])),tweet[z][1])      
        res+=[np.array([x[0] for x in tweet])]
      temp = np.array([np.pad(z.flatten(),(0,Max_input_size*Embedding_size-len(z.flatten()))).reshape(Max_input_size,Embedding_size) for z in res])
      tempres = data_cat[k*(batch_size):(k+1)*(batch_size)]
      yield (temp,tempres)

In [ ]:
epochs = 20

In [ ]:
model_sg = tf.keras.Sequential()
model_sg.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_sg.add(tf.keras.layers.LSTM(50))
model_sg.add(tf.keras.layers.Dense(4, activation='softmax'))
model_sg.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history = model_sg.fit_generator(generator = vec_gen(w2v_sg,vocab_sg,X_train,y_train,batch_size,Max_input_size,Embedding_size,mul_factor),steps_per_epoch=int(len(X_train)/batch_size),epochs=epochs)
history = model_sg.evaluate(x = vec_gen(w2v_sg,vocab_sg,X_test,y_test,batch_size,Max_input_size,Embedding_size,mul_factor),steps=int(len(y_test)/batch_size))
print(history)

result_table[0]=history[1]
print("SG_LSTM")


Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/20
49/49 [==============================] - 3s 56ms/step - loss: 1.3773 - accuracy: 0.3120
Epoch 2/20
49/49 [==============================] - 3s 57ms/step - loss: 1.3750 - accuracy: 0.3153
Epoch 3/20
49/49 [==============================] - 3s 57ms/step - loss: 1.3752 - accuracy: 0.3153
Epoch 4/20
49/49 [==============================] - 3s 58ms/step - loss: 1.3752 - accuracy: 0.3153
Epoch 5/20
49/49 [==============================] - 3s 59ms/step - loss: 1.3752 - accuracy: 0.3153
Epoch 6/20
49/49 [==============================] - 3s 59ms/step - loss: 1.3751 - accuracy: 0.3153
Epoch 7/20
49/49 [==============================] - 3s 59ms/step - loss: 1.3751 - accuracy: 0.3153
Epoch 8/20
49/49 [==============================] - 3s 59ms/step - loss: 1.3750 - accuracy: 0.3153
Epoch 9/20
49/49 [==============================] - 3s 56ms/step - loss: 1.3750 - accuracy: 0.3153
Epoch 10/20
49/49 [==============

In [ ]:
model_sg_bi = tf.keras.Sequential()
model_sg_bi.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_sg_bi.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(50),merge_mode='concat'))
model_sg_bi.add(tf.keras.layers.Dense(4, activation='softmax'))
model_sg_bi.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history = model_sg_bi.fit_generator(generator = vec_gen(w2v_sg,vocab_sg,X_train,y_train,batch_size,Max_input_size,Embedding_size,mul_factor),steps_per_epoch=int(len(X_train)/batch_size),epochs=epochs)
history = model_sg_bi.evaluate(x = vec_gen(w2v_sg,vocab_sg,X_test,y_test,batch_size,Max_input_size,Embedding_size,mul_factor),steps=int(len(y_test)/batch_size))
print(history)

result_table[1]=history[1]
print("SG_BI")

Epoch 1/20
49/49 [==============================] - 3s 59ms/step - loss: 1.3769 - accuracy: 0.3143
Epoch 2/20
49/49 [==============================] - 3s 59ms/step - loss: 1.3750 - accuracy: 0.3153
Epoch 3/20
49/49 [==============================] - 3s 58ms/step - loss: 1.3749 - accuracy: 0.3153
Epoch 4/20
49/49 [==============================] - 3s 59ms/step - loss: 1.3747 - accuracy: 0.3153
Epoch 5/20
49/49 [==============================] - 3s 60ms/step - loss: 1.3745 - accuracy: 0.3153
Epoch 6/20
49/49 [==============================] - 3s 58ms/step - loss: 1.3743 - accuracy: 0.3153
Epoch 7/20
49/49 [==============================] - 3s 58ms/step - loss: 1.3741 - accuracy: 0.3153
Epoch 8/20
49/49 [==============================] - 3s 59ms/step - loss: 1.3736 - accuracy: 0.3153
Epoch 9/20
49/49 [==============================] - 3s 58ms/step - loss: 1.3728 - accuracy: 0.3153
Epoch 10/20
49/49 [==============================] - 3s 57ms/step - loss: 1.3717 - accuracy: 0.3131
Epoch 11/

In [ ]:
model_sg_rnn = tf.keras.Sequential()
model_sg_rnn.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_sg_rnn.add(tf.keras.layers.SimpleRNN(50))
model_sg_rnn.add(tf.keras.layers.Dense(4, activation='softmax'))
model_sg_rnn.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history = model_sg_rnn.fit_generator(generator = vec_gen(w2v_sg,vocab_sg,X_train,y_train,batch_size,Max_input_size,Embedding_size,mul_factor),steps_per_epoch=int(len(X_train)/batch_size),epochs=epochs)
history = model_sg_rnn.evaluate(x = vec_gen(w2v_sg,vocab_sg,X_test,y_test,batch_size,Max_input_size,Embedding_size,mul_factor),steps=int(len(y_test)/batch_size))
print(history)

result_table[2]=history[1]
print("SG_RNN")

Epoch 1/20
49/49 [==============================] - 4s 80ms/step - loss: 1.3766 - accuracy: 0.3118
Epoch 2/20
49/49 [==============================] - 4s 81ms/step - loss: 1.3716 - accuracy: 0.3206
Epoch 3/20
49/49 [==============================] - 4s 80ms/step - loss: 1.3686 - accuracy: 0.3192
Epoch 4/20
49/49 [==============================] - 4s 80ms/step - loss: 1.3714 - accuracy: 0.3182
Epoch 5/20
49/49 [==============================] - 4s 80ms/step - loss: 1.3675 - accuracy: 0.3245
Epoch 6/20
49/49 [==============================] - 4s 80ms/step - loss: 1.3747 - accuracy: 0.3145
Epoch 7/20
49/49 [==============================] - 4s 84ms/step - loss: 1.3693 - accuracy: 0.3180
Epoch 8/20
49/49 [==============================] - 4s 80ms/step - loss: 1.3693 - accuracy: 0.3184
Epoch 9/20
49/49 [==============================] - 4s 80ms/step - loss: 1.3654 - accuracy: 0.3261
Epoch 10/20
49/49 [==============================] - 4s 78ms/step - loss: 1.3650 - accuracy: 0.3229
Epoch 11/

In [ ]:
model_cbow = tf.keras.Sequential()
model_cbow.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_cbow.add(tf.keras.layers.LSTM(50))
model_cbow.add(tf.keras.layers.Dense(4, activation='softmax'))
model_cbow.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history = model_cbow.fit_generator(generator = vec_gen(w2v_cbow,vocab_cbow,X_train,y_train,batch_size,Max_input_size,Embedding_size,mul_factor),steps_per_epoch=int(len(X_train)/batch_size),epochs=epochs)
history = model_cbow.evaluate(x = vec_gen(w2v_cbow,vocab_cbow,X_test,y_test,batch_size,Max_input_size,Embedding_size,mul_factor),steps=int(len(y_test)/batch_size))
print(history)

result_table[3]=history[1]
print("CBOW_LSTM")

Epoch 1/20
49/49 [==============================] - 3s 57ms/step - loss: 1.3770 - accuracy: 0.3131
Epoch 2/20
49/49 [==============================] - 3s 59ms/step - loss: 1.3753 - accuracy: 0.3153
Epoch 3/20
49/49 [==============================] - 3s 61ms/step - loss: 1.3751 - accuracy: 0.3153
Epoch 4/20
49/49 [==============================] - 3s 59ms/step - loss: 1.3752 - accuracy: 0.3153
Epoch 5/20
49/49 [==============================] - 3s 60ms/step - loss: 1.3751 - accuracy: 0.3153
Epoch 6/20
49/49 [==============================] - 3s 57ms/step - loss: 1.3751 - accuracy: 0.3153
Epoch 7/20
49/49 [==============================] - 3s 57ms/step - loss: 1.3751 - accuracy: 0.3153
Epoch 8/20
49/49 [==============================] - 3s 59ms/step - loss: 1.3750 - accuracy: 0.3153
Epoch 9/20
49/49 [==============================] - 3s 58ms/step - loss: 1.3750 - accuracy: 0.3153
Epoch 10/20
49/49 [==============================] - 3s 62ms/step - loss: 1.3750 - accuracy: 0.3153
Epoch 11/

In [ ]:
model_cbow_bi = tf.keras.Sequential()
model_cbow_bi.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_cbow_bi.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(50),merge_mode='concat'))
model_cbow_bi.add(tf.keras.layers.Dense(4, activation='softmax'))
model_cbow_bi.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history = model_cbow_bi.fit_generator(generator = vec_gen(w2v_cbow,vocab_cbow,X_train,y_train,batch_size,Max_input_size,Embedding_size,mul_factor),steps_per_epoch=int(len(X_train)/batch_size),epochs=epochs)
history = model_cbow_bi.evaluate(x = vec_gen(w2v_cbow,vocab_cbow,X_test,y_test,batch_size,Max_input_size,Embedding_size,mul_factor),steps=int(len(y_test)/batch_size))
print(history)

result_table[4]=history[1]
print("CBOW_BI")

Epoch 1/20
49/49 [==============================] - 3s 59ms/step - loss: 1.3770 - accuracy: 0.3124
Epoch 2/20
49/49 [==============================] - 3s 61ms/step - loss: 1.3750 - accuracy: 0.3153
Epoch 3/20
49/49 [==============================] - 3s 58ms/step - loss: 1.3749 - accuracy: 0.3153
Epoch 4/20
49/49 [==============================] - 3s 60ms/step - loss: 1.3748 - accuracy: 0.3153
Epoch 5/20
49/49 [==============================] - 3s 60ms/step - loss: 1.3746 - accuracy: 0.3153
Epoch 6/20
49/49 [==============================] - 3s 58ms/step - loss: 1.3744 - accuracy: 0.3153
Epoch 7/20
49/49 [==============================] - 3s 58ms/step - loss: 1.3742 - accuracy: 0.3153
Epoch 8/20
49/49 [==============================] - 3s 59ms/step - loss: 1.3739 - accuracy: 0.3153
Epoch 9/20
49/49 [==============================] - 3s 59ms/step - loss: 1.3734 - accuracy: 0.3153
Epoch 10/20
49/49 [==============================] - 3s 59ms/step - loss: 1.3730 - accuracy: 0.3145
Epoch 11/

In [ ]:
model_cbow_rnn = tf.keras.Sequential()
model_cbow_rnn.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_cbow_rnn.add(tf.keras.layers.SimpleRNN(50))
model_cbow_rnn.add(tf.keras.layers.Dense(4, activation='softmax'))
model_cbow_rnn.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history = model_cbow_rnn.fit_generator(generator = vec_gen(w2v_cbow,vocab_cbow,X_train,y_train,batch_size,Max_input_size,Embedding_size,mul_factor),steps_per_epoch=int(len(X_train)/batch_size),epochs=epochs)
history = model_cbow_rnn.evaluate(x = vec_gen(w2v_cbow,vocab_cbow,X_test,y_test,batch_size,Max_input_size,Embedding_size,mul_factor),steps=int(len(y_test)/batch_size))
print(history)

result_table[5]=history[1]
print("CBOW_RNN")

Epoch 1/20
49/49 [==============================] - 4s 79ms/step - loss: 1.3781 - accuracy: 0.3110
Epoch 2/20
49/49 [==============================] - 4s 80ms/step - loss: 1.3745 - accuracy: 0.3141
Epoch 3/20
49/49 [==============================] - 4s 81ms/step - loss: 1.3729 - accuracy: 0.3180
Epoch 4/20
49/49 [==============================] - 4s 80ms/step - loss: 1.3723 - accuracy: 0.3151
Epoch 5/20
49/49 [==============================] - 4s 81ms/step - loss: 1.3705 - accuracy: 0.3188
Epoch 6/20
49/49 [==============================] - 4s 81ms/step - loss: 1.3833 - accuracy: 0.3080
Epoch 7/20
49/49 [==============================] - 4s 81ms/step - loss: 1.3698 - accuracy: 0.3088
Epoch 8/20
49/49 [==============================] - 4s 80ms/step - loss: 1.3686 - accuracy: 0.3135
Epoch 9/20
49/49 [==============================] - 4s 80ms/step - loss: 1.3705 - accuracy: 0.3241
Epoch 10/20
49/49 [==============================] - 4s 81ms/step - loss: 1.3678 - accuracy: 0.3173
Epoch 11/

In [ ]:
epochs=25

In [ ]:
model_glove = tf.keras.Sequential()
model_glove.add(tf.keras.Input(shape=(Max_input_size,100)))
model_glove.add(tf.keras.layers.LSTM(50))
model_glove.add(tf.keras.layers.Dense(4, activation='softmax'))
model_glove.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history = model_glove.fit_generator(generator = glove_gen(vocab_glove,X_train,y_train,batch_size,Max_input_size,Embedding_size,mul_factor),steps_per_epoch=int(len(X_train)/batch_size),epochs=epochs)
history = model_glove.evaluate(x = glove_gen(vocab_glove,X_test,y_test,batch_size,Max_input_size,Embedding_size,mul_factor),steps=int(len(y_test)/batch_size))
print(history)

result_table[6]=history[1]
print("GLOVE_LSTM")


Epoch 1/25
49/49 [==============================] - 1s 13ms/step - loss: 1.3765 - accuracy: 0.3108
Epoch 2/25
49/49 [==============================] - 1s 13ms/step - loss: 1.2852 - accuracy: 0.3941
Epoch 3/25
49/49 [==============================] - 1s 13ms/step - loss: 1.2239 - accuracy: 0.4308
Epoch 4/25
49/49 [==============================] - 1s 13ms/step - loss: 1.1756 - accuracy: 0.4522
Epoch 5/25
49/49 [==============================] - 1s 13ms/step - loss: 1.1491 - accuracy: 0.4653
Epoch 6/25
49/49 [==============================] - 1s 13ms/step - loss: 1.1156 - accuracy: 0.4798
Epoch 7/25
49/49 [==============================] - 1s 13ms/step - loss: 1.0735 - accuracy: 0.5041
Epoch 8/25
49/49 [==============================] - 1s 13ms/step - loss: 1.0304 - accuracy: 0.5378
Epoch 9/25
49/49 [==============================] - 1s 13ms/step - loss: 1.0054 - accuracy: 0.5502
Epoch 10/25
49/49 [==============================] - 1s 13ms/step - loss: 0.9895 - accuracy: 0.5651
Epoch 11/

In [ ]:
model_glove_bi = tf.keras.Sequential()
model_glove_bi.add(tf.keras.Input(shape=(Max_input_size,100)))
model_glove_bi.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(50),merge_mode='concat'))
model_glove_bi.add(tf.keras.layers.Dense(4, activation='softmax'))
model_glove_bi.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history = model_glove_bi.fit_generator(generator = glove_gen(vocab_glove,X_train,y_train,batch_size,Max_input_size,Embedding_size,mul_factor),steps_per_epoch=int(len(X_train)/batch_size),epochs=epochs)
history = model_glove_bi.evaluate(x = glove_gen(vocab_glove,X_test,y_test,batch_size,Max_input_size,Embedding_size,mul_factor),steps=int(len(y_test)/batch_size))
print(history)

result_table[7]=history[1]
print("GLOVE_BI")

Epoch 1/25
49/49 [==============================] - 1s 15ms/step - loss: 1.3557 - accuracy: 0.3359
Epoch 2/25
49/49 [==============================] - 1s 15ms/step - loss: 1.2300 - accuracy: 0.4582
Epoch 3/25
49/49 [==============================] - 1s 15ms/step - loss: 1.1090 - accuracy: 0.5316
Epoch 4/25
49/49 [==============================] - 1s 15ms/step - loss: 1.0158 - accuracy: 0.5839
Epoch 5/25
49/49 [==============================] - 1s 15ms/step - loss: 0.9382 - accuracy: 0.6157
Epoch 6/25
49/49 [==============================] - 1s 14ms/step - loss: 0.8700 - accuracy: 0.6551
Epoch 7/25
49/49 [==============================] - 1s 15ms/step - loss: 0.8075 - accuracy: 0.6859
Epoch 8/25
49/49 [==============================] - 1s 15ms/step - loss: 0.7505 - accuracy: 0.7116
Epoch 9/25
49/49 [==============================] - 1s 14ms/step - loss: 0.7013 - accuracy: 0.7327
Epoch 10/25
49/49 [==============================] - 1s 15ms/step - loss: 0.6561 - accuracy: 0.7514
Epoch 11/

In [ ]:
model_glove_rnn = tf.keras.Sequential()
model_glove_rnn.add(tf.keras.Input(shape=(Max_input_size,100)))
model_glove_rnn.add(tf.keras.layers.SimpleRNN(50))
model_glove_rnn.add(tf.keras.layers.Dense(4, activation='softmax'))
model_glove_rnn.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history = model_glove_rnn.fit_generator(generator = glove_gen(vocab_glove,X_train,y_train,batch_size,Max_input_size,Embedding_size,mul_factor),steps_per_epoch=int(len(X_train)/batch_size),epochs=epochs)
history = model_glove_rnn.evaluate(x = glove_gen(vocab_glove,X_test,y_test,batch_size,Max_input_size,Embedding_size,mul_factor),steps=int(len(y_test)/batch_size))
print(history)

result_table[8]=history[1]
print("GLOVE_RNN")

Epoch 1/25
49/49 [==============================] - 2s 45ms/step - loss: 1.3700 - accuracy: 0.3108
Epoch 2/25
49/49 [==============================] - 2s 45ms/step - loss: 1.3131 - accuracy: 0.3784
Epoch 3/25
49/49 [==============================] - 2s 43ms/step - loss: 1.2842 - accuracy: 0.3965
Epoch 4/25
49/49 [==============================] - 2s 43ms/step - loss: 1.2550 - accuracy: 0.4222
Epoch 5/25
49/49 [==============================] - 2s 45ms/step - loss: 1.2244 - accuracy: 0.4359
Epoch 6/25
49/49 [==============================] - 2s 44ms/step - loss: 1.2021 - accuracy: 0.4557
Epoch 7/25
49/49 [==============================] - 2s 46ms/step - loss: 1.1891 - accuracy: 0.4629
Epoch 8/25
49/49 [==============================] - 2s 43ms/step - loss: 1.1747 - accuracy: 0.4831
Epoch 9/25
49/49 [==============================] - 2s 44ms/step - loss: 1.1554 - accuracy: 0.4912
Epoch 10/25
49/49 [==============================] - 2s 44ms/step - loss: 1.1398 - accuracy: 0.5035
Epoch 11/

In [ ]:
print("\tLstm\t\t\tBiLstm\t\t\tRnn")
print("sg",result_table[:3])
print("cbow",result_table[3:6])
print("glove",result_table[6:9])

	Lstm			BiLstm			Rnn
sg [0.3214285671710968, 0.3223809599876404, 0.3214285671710968]
cbow [0.3214285671710968, 0.3271428644657135, 0.32380953431129456]
glove [0.5804761648178101, 0.6904761791229248, 0.3985714316368103]
